In [159]:
%pylab inline 
import pandas as pd
import altair as alt

Populating the interactive namespace from numpy and matplotlib


In [160]:
df = pd.read_csv("autoMLQuantitativeResults - Survey Info.csv")

In [161]:
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0)).reset_index(drop=True)
df.columns.name = None

In [162]:
df.head()

Participant ID Organization: Size Organization: Type Participant: Role  \
0              1                NaN                NaN               NaN   
1              2                NaN                NaN               NaN   
2              3                NaN                NaN               NaN   
3              4                NaN                NaN               NaN   
4              5                NaN                NaN               NaN   

                 Data: Type Programming: Language Programming: Proficiency  \
0                       NaN                     R             4 - Advanced   
1                    Images                Python             4 - Advanced   
2                       NaN                   NaN                      NaN   
3                       NaN                   NaN                      NaN   
4  Structured data (Tables)         Python;R;Java                      NaN   

  Programming: Yr Experience   ML: Proficiency ML: Yr Experience  ...  \
0                         30  3 - Intermediate                 5  ...   
1                          8  3 - Intermediate                 2  ...   
2                        NaN               NaN               NaN  ...   
3                        NaN               NaN               NaN  ...   
4                        NaN               NaN               NaN  ...   

     Azure AutoML  TransmogrifyAI Likert: Transparency  \
0        Aware of  Never Heard of                    3   
1        Aware of  Never Heard of                    1   
2        Aware of  Never Heard of                    4   
3             NaN             NaN                  NaN   
4  Never Heard of  Never Heard of                    4   

  Likert: Interpretability Likert: Customizability Likert: Generalizability  \
0                        3                       4                        4   
1                        2                       2                        4   
2                        5                       4                        3   
3                      NaN                     NaN                      NaN   
4                        4                       5                        4   

  Likert: Completeness Likert: Ease of Use Likert: Effectiveness  \
0                    4                   5                     5   
1                    4                   4                     3   
2                    4                   5                     4   
3                  NaN                 NaN                   NaN   
4                    3                   4                     5   

  Likert: Efficiency  
0                  5  
1                  4  
2                  5  
3                NaN  
4                  5  

[5 rows x 31 columns]

In [172]:
singlechoice = ['Organization: Size','Organization: Type', 'Programming: Proficiency','ML: Proficiency']
numerical = [ 'Programming: Yr Experience','ML: Yr Experience']

multiselect = [ 'Participant: Role', 'Data: Type', 'Programming: Language', 'ML: Problem', 'AutoML: Stage']
likert = ['Likert: Transparency', 'Likert: Interpretability', 'Likert: Customizability', 'Likert: Generalizability','Likert: Completeness','Likert: Ease of Use','Likert: Effectiveness','Likert: Efficiency']

In [ ]:
'AutoML: Stage',
       'AutoSklearn', 'TPOT', 'Auto-Weka', 'Google Cloud AutoML', 'H2O.ai',
       'Amazon Sagemaker', 'DataRobot', 'BigML OptiML', 'MLJar',
       'Azure AutoML', 'TransmogrifyAI'

In [164]:
def convertListToBinarizedCols(df,groupbyCol:list, listCol:str):
    combinedList = groupbyCol + [listCol]
    newdf = df[combinedList]
    newdf = newdf.set_index(groupbyCol)[listCol].str.split(";",expand=True).stack().reset_index().pivot_table(index=['Participant ID'], columns=0, fill_value=0, aggfunc='size').reset_index()
    return newdf

In [178]:
def plotMultiselect(df, colOfInterest:str,colLabel:str):
    computed = convertListToBinarizedCols(df,["Participant ID"],colOfInterest)
    cols = list(computed.columns)
    cols.remove("Participant ID")
    computeddf= computed[cols].aggregate("sum",axis=0).rename("count").reset_index().rename(columns={0:colLabel})
    chart = alt.Chart(computeddf).mark_bar().encode(
        y=alt.Y(colLabel,sort="-x"),
        x=alt.X("count", axis= alt.Axis(tickMinStep=1))
    )
    return chart 

In [179]:
plotMultiselect(df, 'Data: Type', 'Data Type')

alt.Chart(...)

In [180]:
plotMultiselect(df, 'Participant: Role', 'Job Title')

/usr/local/lib/python3.7/site-packages/altair/utils/core.py:109: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  "Defaulting to nominal.".format(typ))


alt.Chart(...)

In [181]:
plotMultiselect(df, 'Programming: Language', 'Prog Language')

alt.Chart(...)

In [182]:
plotMultiselect(df, 'ML: Problem', 'ML Problem')

alt.Chart(...)

In [183]:
plotMultiselect(df, 'AutoML: Stage', 'Stage in which AutoML is used')

alt.Chart(...)

In [170]:
for lfield in likert:
    chart = alt.Chart(df).mark_bar().encode(
        alt.X(lfield, bin=True),
        y='count()',
    )
#     display(chart) 